In [1]:
%%capture
!pip install transformers

In [2]:
# as Data Scientist
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
TRAIN_PATH = '/content/drive/MyDrive/Public/DS102 - Machine Learning/data/train.csv'
DEV_PATH = '/content/drive/MyDrive/Public/DS102 - Machine Learning/data/dev.csv'
TEST_PATH = '/content/drive/MyDrive/Public/DS102 - Machine Learning/data/test.csv'

# job labels
JOB_LABELS_PATH = '/content/drive/MyDrive/Public/DS102 - Machine Learning/data/job_labels.csv'

# Load Data

In [5]:
train = pd.read_csv(TRAIN_PATH)
dev = pd.read_csv(DEV_PATH)
test = pd.read_csv(TEST_PATH)

# job labels
labels = pd.read_csv(JOB_LABELS_PATH)

In [7]:
X_train = train['description'] + ' ' + train['requirement'].fillna('')
X_dev = dev['description'] + ' ' + dev['requirement'].fillna('')
X_test = test['description'] + ' ' + test['requirement'].fillna('')

y_train = train['mapped_industry']
y_dev = dev['mapped_industry']
y_test = test['mapped_industry']

In [8]:
labels = labels['0'].tolist()
num_labels = len(labels)

In [9]:
# Make onehot label
def create_onehot_labels(y):
    full_labels = []

    for val in y:
        seperated = val.split(' / ')
        lbl = np.zeros(num_labels)

        for i in range(0, num_labels):
            if labels[i] in seperated:
                lbl[i] = 1

        full_labels.append(lbl)

    return np.array(full_labels)

In [10]:
y_train_onehot = create_onehot_labels(y_train)
y_dev_onehot = create_onehot_labels(y_dev)
y_test_onehot = create_onehot_labels(y_test)

In [11]:
X_train = np.array(X_train)
X_dev = np.array(X_dev)
X_test = np.array(X_test)

# Hyperparameters

In [12]:
MODEL_NAME = 'bert-base-multilingual-cased'
MAX_SEQUENCE_LENGTH = 200

BATCH_SIZE = 16
EPOCH = 20

# Prepare data

In [13]:
# Tokenizer
from transformers import BertTokenizer, TFBertModel
from tqdm.auto import tqdm
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

In [14]:
train_input_ids = np.zeros((len(X_train), MAX_SEQUENCE_LENGTH))
train_attn_masks = np.zeros((len(X_train), MAX_SEQUENCE_LENGTH))
train_token_type_ids = np.zeros((len(X_train), MAX_SEQUENCE_LENGTH))

dev_input_ids = np.zeros((len(X_dev), MAX_SEQUENCE_LENGTH))
dev_attn_masks = np.zeros((len(X_dev), MAX_SEQUENCE_LENGTH))
dev_token_type_ids = np.zeros((len(X_dev), MAX_SEQUENCE_LENGTH))

test_input_ids = np.zeros((len(X_test), MAX_SEQUENCE_LENGTH))
test_attn_masks = np.zeros((len(X_test), MAX_SEQUENCE_LENGTH))
test_token_type_ids = np.zeros((len(X_test), MAX_SEQUENCE_LENGTH))

In [15]:
def encode_data(text_data, ids, masks, token_type, tokenizer, max_length=MAX_SEQUENCE_LENGTH):
    i = 0
    for txt_data in tqdm(text_data):
        tokenized = tokenizer.encode_plus(
            txt_data,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        )

        ids[i, :] = tokenized.input_ids
        masks[i, :] = tokenized.attention_mask
        token_type[i, :] = tokenized.token_type_ids
        i += 1
    return ids, masks,token_type

In [16]:
# Train
train_input_ids, train_attn_masks, train_token_type_ids = encode_data(
    X_train, train_input_ids, train_attn_masks, train_token_type_ids, tokenizer
)
# Dev
dev_input_ids, dev_attn_masks, dev_token_type_ids = encode_data(
    X_dev, dev_input_ids, dev_attn_masks, dev_token_type_ids, tokenizer
)
# Test
test_input_ids, test_attn_masks, test_token_type_ids = encode_data(
    X_test, test_input_ids, test_attn_masks, test_token_type_ids, tokenizer
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/1568 [00:00<?, ?it/s]

In [17]:
# To tensors
train_dataset = tf.data.Dataset.from_tensor_slices((train_input_ids, train_attn_masks, y_train_onehot))
dev_dataset = tf.data.Dataset.from_tensor_slices((dev_input_ids, dev_attn_masks, y_dev_onehot))
test_dataset = tf.data.Dataset.from_tensor_slices((test_input_ids, test_attn_masks, y_test_onehot))

In [18]:
def dataset_map(input_ids, attn_masks, labels):
    return {'input_ids': input_ids, 'attention_mask': attn_masks}, labels

In [19]:
train_dataset = train_dataset.map(dataset_map)
dev_dataset = dev_dataset.map(dataset_map)
test_dataset = test_dataset.map(dataset_map)

In [20]:
train_dataset = train_dataset.shuffle(len(train_dataset)).batch(BATCH_SIZE, drop_remainder=True)
dev_dataset = dev_dataset.shuffle(len(dev_dataset)).batch(BATCH_SIZE, drop_remainder=True)

# Model

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import Sequential
from tensorflow.keras.layers import (
    Input, Dense, Flatten, Embedding, SpatialDropout1D, LSTM
)

In [22]:
def build_model():

    # Input
    input_ids = Input(shape=(MAX_SEQUENCE_LENGTH,), name='input_ids', dtype='int32')
    attn_masks = Input(shape=(MAX_SEQUENCE_LENGTH,), name='attention_mask', dtype='int32')

    # Embedding
    embedding = TFBertModel.from_pretrained(MODEL_NAME, trainable=False)
    bert_embedded = embedding.bert(input_ids, attn_masks)[1]

    # Hidden layers
    hidden_layer = Dense(768, activation='relu', name='hidden_layer')(bert_embedded)

    # Classification
    output_layer = Dense(num_labels, activation='sigmoid', name='output_layer')(hidden_layer)

    # Define model
    model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)

    # Model setting
    optim = tf.keras.optimizers.Adam(learning_rate=1e-5)#, decay=1e-6)
    loss_func = tf.keras.losses.BinaryCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

    # Compile model
    model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

    return model

def build_model():

    # Input
    input_ids = Input(shape=(MAX_SEQUENCE_LENGTH,), name='input_ids', dtype='int32')
    attn_masks = Input(shape=(MAX_SEQUENCE_LENGTH,), name='attention_mask', dtype='int32')

    # BERT Embedding
    embedding = TFBertModel.from_pretrained(MODEL_NAME, trainable=False)
    bert_embedded = embedding.bert(input_ids, attn_masks)[1]

    #

In [23]:
model = build_model()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w